#Data Exploration and Preprocession
Data set used is a Malawi Secondary Ceritificate of Examination syllabus for the Agriculture subject subject, with close to 200 questions. The set of questions are a summary from over 5 MSCE syllabus books that the Ministry of Education in Malawi approved for the Agriculture subject. This chatbot will help secondary school students prepare for Agriculture examinations and increase the passing of of students taking the Agriculture subject.

In [ ]:
!sudo apt-get install tesseract-ocr -y
!sudo apt-get install poppler-utils -y
%pip install PyMuPDF

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip uninstall transformers -y
!pip uninstall tokenizers -y

Found existing installation: transformers 4.57.0
Uninstalling transformers-4.57.0:
  Successfully uninstalled transformers-4.57.0
Found existing installation: tokenizers 0.22.1
Uninstalling tokenizers-0.22.1:
  Successfully uninstalled tokenizers-0.22.1


In [ ]:
pip install transformers datasets tensorflow

In [5]:
import fitz
import re
import pandas as pd
import json
from datasets import Dataset

# Define the file path for your new agriculture PDF
pdf_file_path = "/content/drive/MyDrive/summative_assignment_chatbot/AGRICULTURE-BOOK-4-Q-A.pdf"

# Function to extract text from a digital PDF
def extract_text_from_pdf(file_path):
    doc = fitz.open(file_path)
    full_text = ""
    for page in doc:
        full_text += page.get_text()
    doc.close()
    return full_text

# Run the function to get the text
raw_text = extract_text_from_pdf(pdf_file_path)

# Print a small portion of the raw text to verify the extraction
print(raw_text[:1000])

# Check the length to see if any text was extracted
print(f"\nTotal characters extracted: {len(raw_text)}")

 
 
Compiled by L.L Mpandason  
1 
MSCE AGRICULTURE 
 
QUESTIONS  
& 
 MODEL 
ANSWER 
 
MPANDASON 
 
 
 
Compiled by L.L Mpandason  
2 
 
LUKE LINGISON MPANDASON 
P. O Box 32 
Phalula  
Balaka  
Cell: 0881 615 761/ 0995 516 420 
Email:  lingisonluke08@gmail.com/ lukempandason@gmail.com 
 
 
Under reference from Longhorn Publishers (greymattermw.com)  
 
 
 
©Luke L. Mpandason  
 
 
 
All rights reserved. No part of this publication may be reproduced, stored in a retrieval system or transmitted in any 
form or by any means, electronic, mechanical, photocopying or otherwise without prior written permission of the 
copyright owner 
 
 
 
First published, 2020 
 
 
ISBN 978 9996 038 617 
 
 
 
 
Compiled by L.L Mpandason  
3 
Topics covered  
1. Agricultural and the environment 
 Soil degradation 
 Agricultural and climate change 
 Land drainage  
2. Agricultural research and technology  
 Farm mechanization  
 Farm power  
 Gender and agricultural technology  
 Improved farming tec

In [6]:
def parse_agriculture_qa(text):
    # Split the text by the question number pattern
    chunks = re.split(r'(\d+\.\s)', text)[1:]

    qa_pairs = []

    # Iterate through the chunks to form Q&A pairs
    for i in range(0, len(chunks), 2):
        question_number = chunks[i].strip()
        content = chunks[i+1].strip()

        # The answer starts on a new line or with a bullet point
        lines = content.split('\n', 1)
        if len(lines) > 1:
            question_text = lines[0].strip()
            answer_text = lines[1].strip()

            qa_pairs.append({
                "question": question_text,
                "context": f"{question_text}\n{answer_text}",
                "answer": answer_text
            })
        else:
            print(f"Warning: Failed to parse Q&A for chunk starting with {question_number}")

    return qa_pairs

# Run the parsing function on your raw text
qa_dataset = parse_agriculture_qa(raw_text)

# Save to a JSON file
qa_df = pd.DataFrame(qa_dataset)
qa_df.to_json('msce_agriculture_qa.json', orient='records', indent=4)

print(f"\nSuccessfully parsed {len(qa_dataset)} Q&A pairs.")
print("\nDataset saved to 'msce_agriculture_qa.json'.")


Successfully parsed 161 Q&A pairs.

Dataset saved to 'msce_agriculture_qa.json'.


In [7]:
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import T5Tokenizer

# Load your structured data from the JSON file
qa_df = pd.read_json('msce_agriculture_qa.json')

# Convert the pandas DataFrame to a Hugging Face Dataset object
dataset = Dataset.from_pandas(qa_df)

# Initialize the T5 tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-small')

# Display the first example to check the format
print(dataset[0])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


{'question': 'Agricultural and the environment', 'context': 'Agricultural and the environment\n\uf0b7 Soil degradation \n\uf0b7 Agricultural and climate change \n\uf0b7 Land drainage', 'answer': '\uf0b7 Soil degradation \n\uf0b7 Agricultural and climate change \n\uf0b7 Land drainage'}


In [8]:
import numpy as pd
def prepare_train_features(examples):
    # This is a prefix that tells the T5 model what task to perform.
    prefix = "question: "

    # Combine the prefix, question, and context for the T5 input.
    inputs = [prefix + q + " context: " + c for q, c in zip(examples["question"], examples["context"])]

    # Tokenize the inputs, returning TensorFlow tensors
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length", return_tensors='tf')

    # Tokenize the answers, which will serve as the labels for the model.
    labels = tokenizer(examples["answer"], max_length=512, truncation=True, padding="max_length", return_tensors='tf')

    # Set the labels for the model.
    model_inputs["labels"] = labels["input_ids"]

    # Convert the TensorFlow tensors to NumPy arrays for the Trainer
    return {k: v.numpy() for k, v in model_inputs.items()}

In [9]:
!pip install wandb -qU

In [10]:
import wandb
import random
import math

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import TFT5ForConditionalGeneration, T5Tokenizer, TrainingArguments

# --- CONFIGURATION (Ensure these match your actual settings) ---
MODEL_NAME = "CeeJay00100/t5-edu-bot"
OUTPUT_DIR = "./agriculture_t5_model"
LEARNING_RATE = 2e-5
BATCH_SIZE = 2
NUM_EPOCHS = 3
MAX_SEQ_LENGTH = 512
JSON_FILE = 'msce_agriculture_qa.json'

# --- 1. DATA LOADING AND TOKENIZER SETUP ---
print("--- Loading Data and Tokenizer ---")
qa_df = pd.read_json(JSON_FILE)
dataset = Dataset.from_pandas(qa_df)
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)

# --- 2. T5 PREPROCESSING FUNCTION ---
def prepare_train_features(examples):
    prefix = "question: "
    inputs = [prefix + q + " context: " + c for q, c in zip(examples["question"], examples["context"])]

    model_inputs = tokenizer(inputs, max_length=MAX_SEQ_LENGTH, truncation=True, padding="max_length", return_tensors='tf')
    labels = tokenizer(examples["answer"], max_length=MAX_SEQ_LENGTH, truncation=True, padding="max_length", return_tensors='tf')

    labels_np = labels["input_ids"].numpy()
    labels_np[labels_np == tokenizer.pad_token_id] = -100
    model_inputs["labels"] = tf.constant(labels_np)

    return {k: v.numpy() for k, v in model_inputs.items() if k != "labels"} | {"labels": tf.constant(labels_np).numpy()}

# --- 3. APPLY PREPROCESSING AND SPLIT DATA ---
print("--- Applying Preprocessing and Splitting Datasets ---")
tokenized_dataset = dataset.map(prepare_train_features, batched=True)
tokenized_dataset.set_format(type='tensorflow', columns=['input_ids', 'attention_mask', 'labels'])

train_test_split_dataset = tokenized_dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split_dataset['train']
eval_dataset = train_test_split_dataset['test']

# --- 4. KERAS TRAINING SETUP ---
print("--- Loading T5 Model ---")
model = TFT5ForConditionalGeneration.from_pretrained(MODEL_NAME)

# Convert Hugging Face datasets to TensorFlow datasets
tf_train_dataset = train_dataset.to_tf_dataset(
    columns=['input_ids', 'attention_mask', 'labels'],
    batch_size=BATCH_SIZE,
    shuffle=True,
)

tf_eval_dataset = eval_dataset.to_tf_dataset(
    columns=['input_ids', 'attention_mask', 'labels'],
    batch_size=BATCH_SIZE,
    shuffle=False,
)

# Compile the model with an optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
model.compile(optimizer=optimizer)

# Fine-tune the model
print("--- Starting Fine-Tuning (Keras model.fit) ---")
history = model.fit(
    tf_train_dataset,
    validation_data=tf_eval_dataset,
    epochs=NUM_EPOCHS,
)

# --- 5. SAVE MODEL ---
print("--- Saving Fine-Tuned Model ---")
model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print(f"Model saved to {OUTPUT_DIR}")

--- Loading Data and Tokenizer ---


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


--- Applying Preprocessing and Splitting Datasets ---


Map:   0%|          | 0/161 [00:00<?, ? examples/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.


--- Loading T5 Model ---


/usr/local/lib/python3.12/dist-packages/tf_keras/src/initializers/initializers.py:121: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at CeeJay00100/t5-edu-bot.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


--- Starting Fine-Tuning (Keras model.fit) ---
Epoch 1/3
64/64 [==============================] - 2904s 44s/step - loss: 0.1394 - val_loss: 0.0044
Epoch 2/3
64/64 [==============================] - 2837s 44s/step - loss: 0.0413 - val_loss: 0.0020
Epoch 3/3
64/64 [==============================] - 2879s 45s/step - loss: 0.0286 - val_loss: 0.0013
--- Saving Fine-Tuned Model ---
Model saved to ./agriculture_t5_model


In [4]:
from google.colab import drive
import shutil
import os

# Mount your Google Drive
drive.mount('/content/drive')

# Define the source and destination paths
# The source is the local directory where your model was saved
source_path = './agriculture_t5_model'

# The destination is the path in your Google Drive
destination_path = '/content/drive/MyDrive/summative_assignment_chatbot/agriculture_t5_model'

# Check if the destination path exists, create it if not
if not os.path.exists(destination_path):
    os.makedirs(destination_path)

# Copy the saved model files from the local directory to Google Drive
print(f"--- Copying model from {source_path} to {destination_path} ---")
shutil.copytree(source_path, destination_path, dirs_exist_ok=True)
print("--- Model successfully copied to Google Drive! ---")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
--- Copying model from ./agriculture_t5_model to /content/drive/MyDrive/summative_assignment_chatbot/agriculture_t5_model ---
--- Model successfully copied to Google Drive! ---
